In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("../dataset/csic_database.csv")
df.sample(5, random_state=44)

,Unnamed: 0,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
57668,Anomalous,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=CBC20B3C950EB08440128718ACD59575,NaN,close,NaN,NaN,1,http://localhost:8080/dvwa/publico/registro.js...
19858,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=97EEB1A3105521CB80F285266DDA0014,NaN,close,NaN,NaN,0,http://localhost:8080/dvwa/miembros/imagenes/c...
8404,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=618FF41E9F5D636EA41AB1C4D36DA30A,NaN,close,NaN,NaN,0,http://localhost:8080/dvwa/publico/vaciar.jsp?...
47895,Anomalous,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=0AA8A3C4F285D4E6152B970325F9A926,application/x-www-form-urlencoded,Connection: close,Content-Length: 41,modo=insertar&precio=4935&B1=Confirmar%2F,1,http://localhost:8080/dvwa/publico/pagar.jsp H...
14255,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=4CD27A4CAB1504A4772F470E88B3298D,NaN,close,NaN,NaN,0,http://localhost:8080/dvwa/imagenes/nuestratie...


In [3]:
# def get_length(row):
#     if row is np.nan:
#         return 0
#     m = re.match('Content-Length: (\d+)$', row)
#     return int(m.group(1)) if m else 0
# df['content_length'] = df.lenght.apply(get_length)

In [4]:
methods = set(df['Method'])
hosts = set(df['host'])
d_methods = { k : v for k,v in zip(methods, range(len(methods))) }
d_hosts = { k : v for k,v in zip(hosts, range(len(hosts))) }

In [5]:
df['method_num'] = df.Method.apply(lambda x : d_methods[x])
df['host_num'] = df.host.apply(lambda x : d_hosts[x])

In [6]:
#df.drop(df.columns[[0]], axis=1, inplace=True)
X = df[['method_num', 'host_num']] #'content_length'
y = df['classification']
X.loc[0:4]

,method_num,host_num
0,0,0
1,0,0
2,1,0
3,0,0
4,1,0


In [7]:
from input_proc import *
df1 = df.URL.apply(lambda row : pd.Series(gen_url_info(row), index=['url_len', 'digits', 'spec_chars', 'not_alpha_chars', 'payload_args']))
X = pd.concat([X, df1], axis=1)
# df2 = df.content.apply(lambda row : pd.Series(gen_content_info(row), index=['c_len', 'c_digits', 'c_spec_chars', 'c_not_alpha_chars', 'c_payload_args']))
# X = pd.concat([X, df1, df2], axis=1)
X.sample(5, random_state=44)

,method_num,host_num,url_len,digits,spec_chars,not_alpha_chars,payload_args
57668,0,0,324,65,33,120,12
19858,0,0,34,0,4,5,0
8404,0,0,42,1,4,8,0
47895,1,0,23,0,3,4,0
14255,0,0,32,0,3,4,0


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
clf=RandomForestClassifier(n_estimators=100)

In [10]:
clf.fit(X_train.values,y_train.values)
y_pred=clf.predict(X_test.values)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8312700596056855


In [11]:
clf.feature_importances_

array([0.14939383, 0.00901869, 0.39930602, 0.0874876 , 0.12201515,
       0.20502273, 0.02775599])

In [12]:
clf.predict([X.iloc[54601].tolist()])

array([1], dtype=int64)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.78      0.85      9002
           1       0.74      0.90      0.81      6265

    accuracy                           0.83     15267
   macro avg       0.83      0.84      0.83     15267
weighted avg       0.85      0.83      0.83     15267



In [ ]:
X.to_csv('out-1.csv')

In [14]:
import pickle
pickle.dump(clf, open('../rf_models/rfmodel-02.pkl', 'wb'))

In [ ]:
loaded_model = pickle.load(open('../rf_models/rfmodel-02.pkl', 'rb'))
loaded_model.score(X_test, y_test)

In [ ]:
X.iloc[38268]

In [ ]:
X.iloc[19858]

In [ ]:
clf.predict([
    [0, 0, 115, 56, 4, 7, 12, 0],
    [1, 0, 0, 87, 9, 13, 25, 0]
])